# Objetivo

Ajustar as etapas avaliadas na exploração. Para as variáveis categóricas, serão priorizadas aquelas já definidas como não correlacionadas pelo teste qui-quadrado. Para as variáveis numéricas, será definida a técnica de imputação, aplicação da análise de correlação e posteriormente a padronização.

# Pacotes e funções

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
import pickle

In [2]:
%run ../99.Funcoes_auxiliares/Funcoes.py

In [3]:
# Carrega o imputer e o padronizador construídos na base de treino

path1 = os.getcwd() + '/imputer_apply.pkl'
path2 = os.getcwd() + '/scaler_SC.pkl'

imputer_load = pickle.load(open(path1, 'rb'))
sc_load = pickle.load(open(path2, 'rb'))

# Leitura da base de treino

In [4]:
dados_full = pd.read_parquet('../0.Avalia_Dados/Dados/treino.parquet')

In [5]:
# Colunas desconsideradas: id, documento, mes_pagamento, divida_restante, per_pago, genero
# Colunas desconsideradas momentaneamente para o problema de regressão: target

colunas_remove = ['id', 'documento', 'mes_pagamento', 'divida_restante', 'per_pago', 'GENERO', 'target']

dados_full.drop(columns=colunas_remove, inplace=True)

# Removendo duplicatas, se houver
dados_full.drop_duplicates(inplace=True)

# Seleção das variáveis categóricas

In [6]:
# Variáveis consideradas no notebook de exploração

colunas_consideradas = ['segmento_veiculo', 'VAR_2', 'VAR_42', 'VAR_44', 'VAR_45', 'VAR_46', 'VAR_47', 'VAR_48', 'VAR_50', 'VAR_51', 'VAR_52', 'VAR_53', 'VAR_55', 'VAR_56','VAR_57', 'VAR_113', 'VAR_256']
cat = dados_full.select_dtypes(include=['object'])

cat = cat[colunas_consideradas]
cat = cat.fillna('Sem_Info')

In [7]:
for i in cat.columns:

    print(cat[i].unique())
    print('===================================')

['leves' 'motos']
['E' 'Sem_Info' 'C' 'D' 'B' 'A']
['Sem_Info' 'S' 'N']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'N' 'S']
['Sem_Info' 'S' 'N']
['Sem_Info' 'N' 'S']
['MUITO LONGE' 'PROXIMO' 'Sem_Info']
['N' 'Sem_Info']


In [7]:
cat = ajusta_categorias(cat)

In [9]:
for i in cat.columns:

    print(cat[i].unique())
    print('===================================')

[0 1]
[4 5 2 3 1 0]
[-1  1  0]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  0  1]
[-1  1  0]
[-1  0  1]
[ 1  0 -1]
[ 0 -1]


In [10]:
cat.head()

,segmento_veiculo,VAR_2,VAR_42,VAR_44,VAR_45,VAR_46,VAR_47,VAR_48,VAR_50,VAR_51,VAR_52,VAR_53,VAR_55,VAR_56,VAR_57,VAR_113,VAR_256
index,,,,,,,,,,,,,,,,,
7,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0
8,0,4,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
17,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,0
19,0,4,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
27,0,4,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0


# Seleção das variáveis numéricas

In [8]:
# Somente as variáveis numéricas e já desconsidera as variáveis sem variabilidade

num = dados_full.select_dtypes(exclude=['object'])
num = num.drop(columns=['VAR_140', 'VAR_141', 'VAR_145', 'VAR_154', 'VAR_155', 'VAR_156', 'VAR_162', 'VAR_163', 'VAR_183', 'VAR_199', 'VAR_215', 'VAR_259', 'VAR_261', 'VAR_263'])
num.head()

,dias_atraso,saldo_vencido,pagamento,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,...,VAR_265,VAR_267,VAR_268,VAR_269,VAR_271,VAR_305,VAR_309,VAR_310,VAR_313,VAR_315
index,,,,,,,,,,,,,,,,,,,,,
7,13,361.51,361.51,25.173,0.0,0.051,NaN,NaN,1.333333,NaN,...,0.283,0.542,0.569,1.0,0.816,0.233,0.0,1.0,0.0,1.0
8,11,873.51,873.51,23.767,NaN,0.101,NaN,NaN,1.083333,NaN,...,1.000,NaN,0.000,1.0,1.000,0.002,1.0,1.0,0.0,1.0
17,19,1242.81,1242.81,47.268,0.0,0.057,0.626,0.708,1.500000,2.471023,...,1.000,0.402,0.429,1.0,0.921,0.005,1.0,1.0,0.0,1.0
19,17,2264.37,2264.37,52.789,NaN,0.247,0.203,NaN,0.833333,1.837035,...,0.050,0.679,0.840,1.0,0.408,0.300,0.0,0.0,0.0,1.0
27,19,1110.75,0.00,39.162,2.0,0.744,0.203,0.193,1.583333,2.471023,...,0.243,0.598,0.876,1.0,0.411,0.333,0.0,0.0,0.0,1.0


In [9]:
# Considera uma amostra de 40% dos dados para a técnica de imputação
# Dias de atraso, saldo vencido e pagamento não possuem valores faltantes

df_amostral = num[num.columns.drop(['dias_atraso', 'saldo_vencido', 'pagamento'])].sample(frac=0.4, random_state=42)

In [13]:
imputer = IterativeImputer(max_iter=5, random_state=42)
imputer.fit(df_amostral)

/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


IterativeImputer(max_iter=5, random_state=42)

In [14]:
# Salva o imputer

path = os.getcwd() + '/imputer_apply.pkl'

with open(path, 'wb') as file:
    pickle.dump(imputer, file)

In [10]:
# num_new = pd.DataFrame(imputer.transform(num[df_amostral.columns]), columns=df_amostral.columns)
# num_new = pd.concat([num_new.reset_index(drop=True), num[['dias_atraso', 'saldo_vencido', 'pagamento']].reset_index(drop=True)], axis=1)

num_new = pd.DataFrame(imputer_load.transform(num[df_amostral.columns]), columns=df_amostral.columns)
num_new = pd.concat([num_new.reset_index(drop=True), num[['dias_atraso', 'saldo_vencido', 'pagamento']].reset_index(drop=True)], axis=1)

num_new.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_269,VAR_271,VAR_305,VAR_309,VAR_310,VAR_313,VAR_315,dias_atraso,saldo_vencido,pagamento
0,25.173,0.000000,0.051,0.177286,0.159157,1.333333,1.714026,1.946443,0.145437,0.092704,...,1.0,0.816,0.233,0.0,1.0,0.0,1.0,13,361.51,361.51
1,23.767,-0.047911,0.101,0.204954,0.252725,1.083333,2.220202,3.043996,0.134252,0.009782,...,1.0,1.000,0.002,1.0,1.0,0.0,1.0,11,873.51,873.51
2,47.268,0.000000,0.057,0.626000,0.708000,1.500000,2.471023,7.406250,0.124000,0.054000,...,1.0,0.921,0.005,1.0,1.0,0.0,1.0,19,1242.81,1242.81
3,52.789,0.026722,0.247,0.203000,0.218005,0.833333,1.837035,2.680465,0.094348,0.011000,...,1.0,0.408,0.300,0.0,0.0,0.0,1.0,17,2264.37,2264.37
4,39.162,2.000000,0.744,0.203000,0.193000,1.583333,2.471023,2.867279,0.124000,0.043000,...,1.0,0.411,0.333,0.0,0.0,0.0,1.0,19,1110.75,0.00


## Filtro de correlação

In [11]:
correl = Vars_Correl(num_new, num_new.columns, limiar=0.6)
lista_num_remove = list(set(correl['Var2'].unique()))
num_new = num_new.drop(columns=lista_num_remove)
num_new.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_265,VAR_267,VAR_268,VAR_269,VAR_305,VAR_309,VAR_310,VAR_313,dias_atraso,pagamento
0,25.173,0.000000,0.051,0.177286,0.159157,1.333333,1.714026,1.946443,0.145437,0.092704,...,0.283,0.542000,0.569,1.0,0.233,0.0,1.0,0.0,13,361.51
1,23.767,-0.047911,0.101,0.204954,0.252725,1.083333,2.220202,3.043996,0.134252,0.009782,...,1.000,0.344464,0.000,1.0,0.002,1.0,1.0,0.0,11,873.51
2,47.268,0.000000,0.057,0.626000,0.708000,1.500000,2.471023,7.406250,0.124000,0.054000,...,1.000,0.402000,0.429,1.0,0.005,1.0,1.0,0.0,19,1242.81
3,52.789,0.026722,0.247,0.203000,0.218005,0.833333,1.837035,2.680465,0.094348,0.011000,...,0.050,0.679000,0.840,1.0,0.300,0.0,0.0,0.0,17,2264.37
4,39.162,2.000000,0.744,0.203000,0.193000,1.583333,2.471023,2.867279,0.124000,0.043000,...,0.243,0.598000,0.876,1.0,0.333,0.0,0.0,0.0,19,0.00


## Aplicação da padronização

In [12]:
# O saldo vencido foi adicionado na base novamente, para depois ser realizado o cálculo de percentual pago estimado!

df_num_treino = Padronizacao(num_new.drop(['pagamento'], axis=1), num_new.drop(['pagamento'], axis=1).columns, tipo='padro', nome_sclr = 'scaler_SC.pkl')[0]
df_num_treino = pd.concat([df_num_treino, num_new['pagamento'].reset_index(drop=True)], axis=1)
df_num_treino.head()

O StandardScaler será salvo no caminho: /home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/1.Estudo_Variaveis/scaler_SC.pkl


,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_265,VAR_267,VAR_268,VAR_269,VAR_305,VAR_309,VAR_310,VAR_313,dias_atraso,pagamento
0,-1.347949,-0.471274,-1.359263,-0.477706,-0.724364,-0.008872,-0.438597,-0.504535,-0.392335,0.181183,...,-1.736631,-0.193504,-0.209584,0.539103,0.426519,-0.685574,0.645016,-0.38225,-1.315661,361.51
1,-1.458937,-0.550343,-1.098099,-0.168146,-0.038339,-0.172108,0.468780,0.403462,-0.454944,-0.608823,...,0.610492,-1.149419,-2.453889,0.539103,-0.520164,1.508014,0.645016,-0.38225,-1.394862,873.51
2,0.396197,-0.471274,-1.327924,4.542825,3.299658,0.099952,0.918406,4.012318,-0.512334,-0.187552,...,0.610492,-0.870991,-0.761785,0.539103,-0.507869,1.508014,0.645016,-0.38225,-1.078058,1242.81
3,0.832017,-0.427173,-0.335498,-0.190003,-0.292901,-0.335344,-0.218089,0.102716,-0.678319,-0.597216,...,-2.499364,0.469466,0.859321,0.539103,0.701098,-0.685574,-1.595682,-0.38225,-1.157259,2264.37
4,-0.243678,2.829411,2.260478,-0.190003,-0.476231,0.154364,0.918406,0.257265,-0.512334,-0.292350,...,-1.867572,0.077491,1.001315,0.539103,0.836338,-0.685574,-1.595682,-0.38225,-1.078058,0.00


# Base de treino pré-processada final

In [13]:
df_treino = pd.concat([df_num_treino.reset_index(drop=True), cat.reset_index(drop=True), num['saldo_vencido'].reset_index(drop=True)], axis=1)
df_treino.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_50,VAR_51,VAR_52,VAR_53,VAR_55,VAR_56,VAR_57,VAR_113,VAR_256,saldo_vencido
0,-1.347949,-0.471274,-1.359263,-0.477706,-0.724364,-0.008872,-0.438597,-0.504535,-0.392335,0.181183,...,-1,-1,-1,-1,-1,-1,-1,1,0,361.51
1,-1.458937,-0.550343,-1.098099,-0.168146,-0.038339,-0.172108,0.468780,0.403462,-0.454944,-0.608823,...,0,0,0,0,0,1,0,1,0,873.51
2,0.396197,-0.471274,-1.327924,4.542825,3.299658,0.099952,0.918406,4.012318,-0.512334,-0.187552,...,-1,-1,-1,-1,-1,-1,-1,1,0,1242.81
3,0.832017,-0.427173,-0.335498,-0.190003,-0.292901,-0.335344,-0.218089,0.102716,-0.678319,-0.597216,...,0,0,1,0,0,0,0,1,0,2264.37
4,-0.243678,2.829411,2.260478,-0.190003,-0.476231,0.154364,0.918406,0.257265,-0.512334,-0.292350,...,0,0,0,0,0,0,0,1,0,1110.75


In [14]:
df_treino.isnull().sum().sum()  

np.int64(0)

In [15]:
df_treino.columns

Index(['IDADE', 'VAR_4', 'VAR_5', 'VAR_7', 'VAR_8', 'VAR_9', 'VAR_11',
       'VAR_12', 'VAR_15', 'VAR_17', 'VAR_21', 'VAR_22', 'VAR_23', 'VAR_32',
       'VAR_33', 'VAR_35', 'VAR_40', 'VAR_138', 'VAR_139', 'VAR_142',
       'VAR_143', 'VAR_146', 'VAR_147', 'VAR_149', 'VAR_150', 'VAR_151',
       'VAR_157', 'VAR_158', 'VAR_159', 'VAR_160', 'VAR_170', 'VAR_173',
       'VAR_174', 'VAR_176', 'VAR_178', 'VAR_179', 'VAR_180', 'VAR_188',
       'VAR_210', 'VAR_222', 'VAR_232', 'VAR_253', 'VAR_254', 'VAR_264',
       'VAR_265', 'VAR_267', 'VAR_268', 'VAR_269', 'VAR_305', 'VAR_309',
       'VAR_310', 'VAR_313', 'dias_atraso', 'pagamento', 'segmento_veiculo',
       'VAR_2', 'VAR_42', 'VAR_44', 'VAR_45', 'VAR_46', 'VAR_47', 'VAR_48',
       'VAR_50', 'VAR_51', 'VAR_52', 'VAR_53', 'VAR_55', 'VAR_56', 'VAR_57',
       'VAR_113', 'VAR_256', 'saldo_vencido'],
      dtype='object')

In [16]:
len(df_treino.columns)

72

In [64]:
# path = os.getcwd() + '/colunas_treino.txt'
# np.savetxt(path, df_treino.columns.values, fmt='%s', delimiter=',')

# Base de validação

In [17]:
sc_load.feature_names_in_

array(['IDADE', 'VAR_4', 'VAR_5', 'VAR_7', 'VAR_8', 'VAR_9', 'VAR_11',
       'VAR_12', 'VAR_15', 'VAR_17', 'VAR_21', 'VAR_22', 'VAR_23',
       'VAR_32', 'VAR_33', 'VAR_35', 'VAR_40', 'VAR_138', 'VAR_139',
       'VAR_142', 'VAR_143', 'VAR_146', 'VAR_147', 'VAR_149', 'VAR_150',
       'VAR_151', 'VAR_157', 'VAR_158', 'VAR_159', 'VAR_160', 'VAR_170',
       'VAR_173', 'VAR_174', 'VAR_176', 'VAR_178', 'VAR_179', 'VAR_180',
       'VAR_188', 'VAR_210', 'VAR_222', 'VAR_232', 'VAR_253', 'VAR_254',
       'VAR_264', 'VAR_265', 'VAR_267', 'VAR_268', 'VAR_269', 'VAR_305',
       'VAR_309', 'VAR_310', 'VAR_313', 'dias_atraso'], dtype=object)

In [18]:
imputer_load.feature_names_in_

array(['IDADE', 'VAR_4', 'VAR_5', 'VAR_7', 'VAR_8', 'VAR_9', 'VAR_11',
       'VAR_12', 'VAR_15', 'VAR_17', 'VAR_18', 'VAR_21', 'VAR_22',
       'VAR_23', 'VAR_32', 'VAR_33', 'VAR_34', 'VAR_35', 'VAR_40',
       'VAR_134', 'VAR_138', 'VAR_139', 'VAR_142', 'VAR_143', 'VAR_144',
       'VAR_146', 'VAR_147', 'VAR_148', 'VAR_149', 'VAR_150', 'VAR_151',
       'VAR_152', 'VAR_153', 'VAR_157', 'VAR_158', 'VAR_159', 'VAR_160',
       'VAR_161', 'VAR_164', 'VAR_165', 'VAR_166', 'VAR_167', 'VAR_168',
       'VAR_169', 'VAR_170', 'VAR_171', 'VAR_172', 'VAR_173', 'VAR_174',
       'VAR_175', 'VAR_176', 'VAR_177', 'VAR_178', 'VAR_179', 'VAR_180',
       'VAR_181', 'VAR_182', 'VAR_184', 'VAR_185', 'VAR_186', 'VAR_187',
       'VAR_188', 'VAR_189', 'VAR_190', 'VAR_191', 'VAR_192', 'VAR_193',
       'VAR_194', 'VAR_195', 'VAR_196', 'VAR_197', 'VAR_198', 'VAR_200',
       'VAR_201', 'VAR_202', 'VAR_203', 'VAR_204', 'VAR_205', 'VAR_206',
       'VAR_207', 'VAR_208', 'VAR_209', 'VAR_210', 'VAR_211', 'VA

In [19]:
# Leitura da base de validação 

dados_val = pd.read_parquet('../0.Avalia_Dados/Dados/validacao.parquet')

colunas_remove = ['id', 'documento', 'mes_pagamento', 'divida_restante', 'per_pago', 'GENERO', 'target']

dados_val1 = dados_val[colunas_remove].reset_index(drop=True)

In [20]:
num = pd.DataFrame(imputer_load.transform(dados_val[imputer_load.feature_names_in_]), columns=imputer_load.feature_names_in_)
num = pd.concat([num.reset_index(drop=True), dados_val[['dias_atraso', 'saldo_vencido', 'pagamento']].reset_index(drop=True)], axis=1)
num.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_269,VAR_271,VAR_305,VAR_309,VAR_310,VAR_313,VAR_315,dias_atraso,saldo_vencido,pagamento
0,43.660,0.0,0.558,0.496,0.465000,1.333333,1.056213,1.067279,0.497000,0.345,...,0.400000,0.508000,0.667,0.0,0.0,0.0,1.0,24,693.90,693.90
1,39.121,0.0,0.118,0.176,0.465000,1.166667,2.471023,2.220704,0.124000,0.011,...,0.922102,0.574899,0.006,1.0,1.0,0.0,1.0,68,1926.09,0.00
2,57.151,0.0,0.247,0.203,0.195705,1.500000,2.471023,3.288976,0.034061,0.021,...,1.000000,0.857000,0.667,0.0,0.0,0.0,1.0,21,857.62,857.62
3,45.288,1.0,0.335,0.203,0.182000,1.000000,1.837035,2.727553,0.124000,0.011,...,0.130000,0.603000,0.004,0.0,0.0,1.0,2.0,33,743.02,0.00
4,53.345,0.0,0.405,0.203,0.182000,7.250000,1.837035,2.727553,0.124000,0.011,...,1.000000,0.430000,0.002,0.0,1.0,0.0,1.0,75,2840.13,0.00


In [21]:
num1 = pd.DataFrame(sc_load.transform(num[sc_load.get_feature_names_out()]), columns=sc_load.get_feature_names_out())
num = pd.concat([num1.reset_index(drop=True), num[['saldo_vencido', 'pagamento']].reset_index(drop=True)], axis=1)
num.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_267,VAR_268,VAR_269,VAR_305,VAR_309,VAR_310,VAR_313,dias_atraso,saldo_vencido,pagamento
0,0.111387,-0.471274,1.288945,3.088291,1.518025,-0.008872,-1.617801,-1.231859,1.575627,2.584824,...,-0.087041,-0.761785,-1.844065,2.205135,-0.685574,-1.595682,-0.382250,-0.880056,693.90,693.90
1,-0.246915,-0.471274,-1.009303,-0.492098,1.518025,-0.117696,0.918406,-0.277641,-0.512334,-0.597216,...,0.111366,-0.005899,0.229695,-0.503771,1.508014,0.645016,-0.382250,0.862365,1926.09,0.00
2,1.176346,-0.471274,-0.335498,-0.190003,-0.456397,0.099952,0.918406,0.606132,-1.015788,-0.501945,...,-0.067685,-0.391022,0.539103,2.205135,-0.685574,-1.595682,-0.382250,-0.998857,857.62,857.62
3,0.239899,1.179068,0.124151,-0.190003,-0.556881,-0.226520,-0.218089,0.141671,-0.512334,-0.597216,...,-0.091881,-1.712361,-2.916491,-0.511967,-0.685574,-1.595682,2.080324,-0.523651,743.02,0.00
4,0.875906,-0.471274,0.489782,-0.190003,-0.556881,3.854373,-0.218089,0.141671,-0.512334,-0.597216,...,0.130722,0.973705,0.539103,-0.520164,-0.685574,0.645016,-0.382250,1.139569,2840.13,0.00


In [22]:
colunas_consideradas = ['segmento_veiculo', 'VAR_2', 'VAR_42', 'VAR_44', 'VAR_45', 'VAR_46', 'VAR_47', 'VAR_48', 'VAR_50', 'VAR_51', 'VAR_52', 'VAR_53', 'VAR_55', 'VAR_56','VAR_57', 'VAR_113', 'VAR_256']
cat = ajusta_categorias(dados_val[colunas_consideradas].fillna('Sem_Info')).reset_index(drop=True)
cat.head()

,segmento_veiculo,VAR_2,VAR_42,VAR_44,VAR_45,VAR_46,VAR_47,VAR_48,VAR_50,VAR_51,VAR_52,VAR_53,VAR_55,VAR_56,VAR_57,VAR_113,VAR_256
0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,4,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,0,4,1,1,0,1,0,0,0,0,0,1,0,1,1,0,0
3,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,4,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [23]:
df_validacao = pd.concat([dados_val1[['id', 'documento', 'mes_pagamento']], num, cat], axis=1)
df_validacao.head()

,id,documento,mes_pagamento,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,...,VAR_48,VAR_50,VAR_51,VAR_52,VAR_53,VAR_55,VAR_56,VAR_57,VAR_113,VAR_256
0,0x1b485a,0x1dabfd,202209,0.111387,-0.471274,1.288945,3.088291,1.518025,-0.008872,-1.617801,...,0,0,0,0,0,0,0,0,0,0
1,0x6fc044,0x94cbb,202207,-0.246915,-0.471274,-1.009303,-0.492098,1.518025,-0.117696,0.918406,...,0,0,0,0,0,0,1,0,1,0
2,0x80a6ed,0x7b09ad,202206,1.176346,-0.471274,-0.335498,-0.190003,-0.456397,0.099952,0.918406,...,0,0,0,0,1,0,1,1,0,0
3,0x74c722,0x809089,202205,0.239899,1.179068,0.124151,-0.190003,-0.556881,-0.226520,-0.218089,...,0,0,0,0,0,0,0,0,1,0
4,0x1eddcf,0x1d2dbb,202205,0.875906,-0.471274,0.489782,-0.190003,-0.556881,3.854373,-0.218089,...,0,0,0,0,0,0,0,0,1,0


In [24]:
df_validacao.isnull().sum().sum()

np.int64(0)

# Base out of time

In [25]:
# Leitura da base de teste/oot

dados_oot = pd.read_parquet('../0.Avalia_Dados/Dados/teste.parquet')

colunas_remove = ['id', 'documento', 'mes_pagamento', 'divida_restante', 'per_pago', 'GENERO', 'target']

dados_oot1 = dados_oot[colunas_remove].reset_index(drop=True)

In [26]:
num = pd.DataFrame(imputer_load.transform(dados_oot[imputer_load.feature_names_in_]), columns=imputer_load.feature_names_in_)
num = pd.concat([num.reset_index(drop=True), dados_oot[['dias_atraso', 'saldo_vencido', 'pagamento']].reset_index(drop=True)], axis=1)
num.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_269,VAR_271,VAR_305,VAR_309,VAR_310,VAR_313,VAR_315,dias_atraso,saldo_vencido,pagamento
0,42.273,0.285329,0.311909,0.219993,0.258023,1.350873,1.958049,2.560870,0.214867,0.073632,...,0.863743,0.593448,0.129102,0.313028,0.71182,0.153243,1.161673,18,1445.37,0.00
1,42.273,0.285329,0.311909,0.219993,0.258023,1.350873,1.958049,2.560870,0.214867,0.073632,...,0.863743,0.593448,0.129102,0.313028,0.71182,0.153243,1.161673,28,426.84,426.84
2,42.273,0.285329,0.311909,0.219993,0.258023,1.350873,1.958049,2.560870,0.214867,0.073632,...,0.863743,0.593448,0.129102,0.313028,0.71182,0.153243,1.161673,30,1381.98,1381.98
3,55.140,0.000000,0.203000,0.203000,0.202937,0.750000,2.471023,3.348282,0.093026,0.021000,...,0.195000,0.563000,0.200000,0.000000,1.00000,0.118914,1.117773,22,1299.22,1299.22
4,46.405,0.000000,0.223000,0.203000,0.182000,0.916667,2.471023,2.727553,0.124000,0.011000,...,0.380000,0.481000,0.003000,1.000000,1.00000,0.000000,1.000000,17,493.42,0.00


In [27]:
num1 = pd.DataFrame(sc_load.transform(num[sc_load.get_feature_names_out()]), columns=sc_load.get_feature_names_out())
num = pd.concat([num1.reset_index(drop=True), num[['saldo_vencido', 'pagamento']].reset_index(drop=True)], axis=1)
num.head()

,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,...,VAR_267,VAR_268,VAR_269,VAR_305,VAR_309,VAR_310,VAR_313,dias_atraso,saldo_vencido,pagamento
0,0.001899,-0.000383,0.003542,0.000123,0.000509,0.002580,-0.001158,0.003775,-0.003681,-0.000516,...,0.000391,0.002832,-0.002102,0.000727,0.001080,-0.000709,-0.004876,-1.117659,1445.37,0.00
1,0.001899,-0.000383,0.003542,0.000123,0.000509,0.002580,-0.001158,0.003775,-0.003681,-0.000516,...,0.000391,0.002832,-0.002102,0.000727,0.001080,-0.000709,-0.004876,-0.721654,426.84,426.84
2,0.001899,-0.000383,0.003542,0.000123,0.000509,0.002580,-0.001158,0.003775,-0.003681,-0.000516,...,0.000391,0.002832,-0.002102,0.000727,0.001080,-0.000709,-0.004876,-0.642453,1381.98,1381.98
3,1.017601,-0.471274,-0.565323,-0.190003,-0.403375,-0.389755,0.918406,0.655195,-0.685721,-0.501945,...,-0.091881,-1.448093,-2.658314,0.291279,-0.685574,0.645016,-0.089416,-0.959257,1299.22,1299.22
4,0.328073,-0.471274,-0.460857,-0.190003,-0.556881,-0.280931,0.918406,0.141671,-0.512334,-0.597216,...,-1.204896,-0.793340,-1.923504,-0.516066,1.508014,0.645016,-0.382250,-1.157259,493.42,0.00


In [28]:
colunas_consideradas = ['segmento_veiculo', 'VAR_2', 'VAR_42', 'VAR_44', 'VAR_45', 'VAR_46', 'VAR_47', 'VAR_48', 'VAR_50', 'VAR_51', 'VAR_52', 'VAR_53', 'VAR_55', 'VAR_56','VAR_57', 'VAR_113', 'VAR_256']
cat = ajusta_categorias(dados_oot[colunas_consideradas].fillna('Sem_Info')).reset_index(drop=True)
cat.head()

,segmento_veiculo,VAR_2,VAR_42,VAR_44,VAR_45,VAR_46,VAR_47,VAR_48,VAR_50,VAR_51,VAR_52,VAR_53,VAR_55,VAR_56,VAR_57,VAR_113,VAR_256
0,0,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,4,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [29]:
df_oot = pd.concat([dados_oot1[['id', 'documento', 'mes_pagamento']], num, cat], axis=1)
df_oot.head()

,id,documento,mes_pagamento,IDADE,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,...,VAR_48,VAR_50,VAR_51,VAR_52,VAR_53,VAR_55,VAR_56,VAR_57,VAR_113,VAR_256
0,0x117651,0x722945,202301,0.001899,-0.000383,0.003542,0.000123,0.000509,0.002580,-0.001158,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0x80136d,0x7fa1e8,202301,0.001899,-0.000383,0.003542,0.000123,0.000509,0.002580,-0.001158,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0x839c27,0x724e31,202301,0.001899,-0.000383,0.003542,0.000123,0.000509,0.002580,-0.001158,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0x7675af,0x827ea9,202210,1.017601,-0.471274,-0.565323,-0.190003,-0.403375,-0.389755,0.918406,...,0,0,0,0,0,0,0,0,0,0
4,0x52fa00,0x2772b2,202210,0.328073,-0.471274,-0.460857,-0.190003,-0.556881,-0.280931,0.918406,...,0,0,0,0,0,1,0,0,1,0


In [50]:
df_oot.isnull().sum().sum()

np.int64(0)

# Salva as bases pré-processadas

In [30]:
df_treino.to_parquet('../0.Avalia_Dados/Dados/treino_pp.parquet', engine='fastparquet')
df_validacao.to_parquet('../0.Avalia_Dados/Dados/validacao_pp.parquet', engine='fastparquet')
df_oot.to_parquet('../0.Avalia_Dados/Dados/oot_pp.parquet', engine='fastparquet')

# Conclusão

- Bases de treino, validação e teste (oot) pré-processadas. A base de treino é a única que não permaneceu com as informações de id, documento e mes de pagamento.